In [1]:
import os
import sys
import pickle
import numpy as np
import random
import copy
sys.path.insert(0, os.getcwd())

os.chdir('../..')
from lib.utils.tools import read_pkl
from lib.data.datareader_h36m import DataReaderH36M
from lib.data.datareader_aihub import DataReaderAIHUB
from tqdm import tqdm

In [2]:
pwd

'/home/hrai/codes/MotionBERT'

In [3]:
#datareader = DataReaderH36M(n_frames=243, sample_stride=1, data_stride_train=81, data_stride_test=243, dt_file = 'h36m_sh_conf_cam_source_final.pkl', dt_root='data/motion3d/')
datareader = DataReaderAIHUB(n_frames=243, sample_stride=1, data_stride_train=81, data_stride_test=243, dt_file = 'aihub_30.pkl', dt_root='data/motion3d')


In [13]:
vid_list_train = datareader.dt_dataset['train']['source'][::datareader.sample_stride]                          # (1559752,)
vid_list_test = datareader.dt_dataset['test']['source'][::datareader.sample_stride]                           # (566920,)

In [16]:
vid_list_train.shape, vid_list_test.shape

((12904,), (3407,))

In [18]:
from lib.utils.utils_data import split_clips

In [38]:
datareader.split_id_train = split_clips(vid_list_train, datareader.n_frames, data_stride=datareader.data_stride_train) 

In [39]:
datareader.split_id_test = split_clips(vid_list_test, datareader.n_frames, data_stride=datareader.data_stride_test)

In [44]:
datareader.split_id_test[-1]

array([2957, 2958, 2958, 2959, 2961, 2962, 2962, 2964, 2964, 2966, 2966,
       2967, 2969, 2969, 2969, 2971, 2971, 2973, 2973, 2974, 2976, 2976,
       2977, 2978, 2979, 2981, 2981, 2982, 2982, 2983, 2985, 2986, 2987,
       2988, 2988, 2990, 2990, 2991, 2992, 2994, 2995, 2995, 2996, 2996,
       2998, 2998, 3000, 3000, 3002, 3002, 3003, 3004, 3005, 3006, 3007,
       3008, 3008, 3009, 3010, 3011, 3012, 3013, 3015, 3015, 3017, 3018,
       3018, 3019, 3020, 3020, 3022, 3023, 3023, 3025, 3025, 3026, 3028,
       3029, 3029, 3031, 3032, 3032, 3032, 3034, 3034, 3035, 3037, 3038,
       3038, 3039, 3040, 3041, 3042, 3043, 3044, 3045, 3045, 3046, 3048,
       3048, 3049, 3050, 3051, 3053, 3054, 3055, 3055, 3057, 3057, 3057,
       3058, 3060, 3060, 3061, 3062, 3063, 3064, 3065, 3067, 3068, 3068,
       3070, 3070, 3070, 3071, 3072, 3074, 3074, 3076, 3076, 3078, 3078,
       3080, 3080, 3082, 3082, 3083, 3084, 3084, 3085, 3087, 3088, 3089,
       3089, 3090, 3092, 3092, 3093, 3094, 3094, 30

In [4]:
train_data, test_data, train_labels, test_labels = datareader.get_sliced_data()

In [4]:
train_data, test_data = datareader.read_2d()     # train_data (1559752, 17, 3) test_data (566920, 17, 3)
train_labels, test_labels = datareader.read_3d() # train_labels (1559752, 17, 3) test_labels (566920, 17, 3)

In [7]:
type(train_data), train_data.shape, type(test_data), test_data.shape

(numpy.ndarray, (12904, 17, 3), numpy.ndarray, (3407, 17, 3))

In [9]:
split_id_train, split_id_test = datareader.get_split_id()
len(split_id_train), len(split_id_test)

(84, 11)

In [10]:
type(split_id_train), type(split_id_test)

(list, list)

In [11]:
train_data.shape, test_data.shape

((12904, 17, 3), (3407, 17, 3))

In [37]:
test_data

array([[[ 0.11796939, -0.03744149,  0.9710206 ],
        [ 0.0978415 , -0.03240955,  0.8329151 ],
        [ 0.0978415 ,  0.08835721,  0.95069426],
        ...,
        [ 0.07771373, -0.17833608,  0.94025815],
        [ 0.0626179 , -0.0978249 ,  0.898675  ],
        [ 0.07771373, -0.03240955,  0.9391023 ]],

       [[ 0.12249148, -0.0377472 ,  0.96175873],
        [ 0.09738421, -0.03272575,  0.84564275],
        [ 0.09738421,  0.09281057,  0.9526451 ],
        ...,
        [ 0.0772984 , -0.17834789,  0.9452678 ],
        [ 0.06223404, -0.09800464,  0.8988067 ],
        [ 0.0772984 , -0.0277043 ,  0.93647885]],

       [[ 0.11910331, -0.03805459,  0.9468791 ],
        [ 0.09396923, -0.03302771,  0.88666093],
        [ 0.09899604,  0.09264314,  0.96426105],
        ...,
        [ 0.07386196, -0.17880595,  0.9520674 ],
        [ 0.0638082 , -0.0983766 ,  0.9209594 ],
        [ 0.07386196, -0.02800089,  0.932181  ]],

       ...,

       [[ 0.22565341,  0.03929365,  0.90547675],
        [ 0

In [36]:
test_data[split_id_train[:10]]

/tmp/ipykernel_10752/3966067609.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  test_data[split_id_train[:10]]


IndexError: too many indices for array: array is 3-dimensional, but 10 were indexed

In [8]:
train_data = copy.deepcopy(train_data[split_id_train])
train_data.shape

(84, 243, 17, 3)

In [39]:
np.array([test_data[split_id_test[i]] for i in range(len(split_id_test))])

(11, 243, 17, 3)

In [9]:
train_labels, test_labels = train_labels[split_id_train], test_labels[split_id_test]

/tmp/ipykernel_10662/1080642337.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  train_labels, test_labels = train_labels[split_id_train], test_labels[split_id_test]


IndexError: too many indices for array: array is 3-dimensional, but 11 were indexed

In [22]:
test_data =  copy.deepcopy(test_data[split_id_test])
test_data.shape

/tmp/ipykernel_8963/2835331642.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  test_data =  copy.deepcopy(test_data[split_id_test])


IndexError: too many indices for array: array is 3-dimensional, but 11 were indexed

/tmp/ipykernel_8209/3011234132.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  test_data =  test_data[split_id_test]


IndexError: index 225 is out of bounds for axis 1 with size 17

In [27]:
split_id_test

[array([  0,   0,   1,   2,   4,   5,   6,   7,   7,   8,  10,  11,  11,
         12,  12,  13,  14,  15,  16,  17,  18,  19,  21,  22,  23,  23,
         24,  25,  25,  27,  28,  28,  29,  30,  32,  33,  34,  34,  36,
         37,  37,  37,  39,  39,  41,  42,  42,  43,  45,  45,  46,  48,
         48,  50,  50,  50,  51,  52,  54,  54,  56,  57,  58,  58,  60,
         60,  62,  62,  63,  63,  64,  65,  66,  68,  68,  69,  71,  71,
         72,  74,  75,  75,  75,  76,  78,  78,  80,  81,  81,  82,  84,
         85,  86,  86,  88,  88,  88,  90,  90,  92,  93,  94,  95,  95,
         97,  98,  98, 100, 100, 101, 101, 102, 104, 104, 105, 106, 108,
        108, 109, 111, 112, 113, 113, 114, 114, 115, 117, 118, 119, 119,
        120, 122, 123, 123, 124, 125, 126, 127, 127, 128, 129, 130, 131,
        132, 134, 134, 136, 136, 138, 138, 138, 139, 140, 141, 143, 144,
        144, 145, 147, 148, 148, 149, 150, 150, 151, 153, 154, 154, 156,
        156, 158, 159, 160, 161, 162, 162, 163, 163

In [33]:
test_data.shape

(675, 17, 3)

In [32]:
test_data[split_id_test]

/tmp/ipykernel_8209/1774937449.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  test_data[split_id_test]


IndexError: index 225 is out of bounds for axis 1 with size 17